In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = openai_api_key

tavily_api_key = os.getenv("TAVILY_API_KEY")
os.environ["TAVILY_API_KEY"] = tavily_api_key

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator, sqlite3
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver

conn = sqlite3.connect("checkpoints.sqlite", check_same_thread=False)
memory = SqliteSaver(conn)

In [3]:
from uuid import uuid4
from langchain_core.messages import AIMessage

"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [4]:
tool = TavilySearchResults(max_results=2)

In [5]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"] # added interupt before action, to take human response
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [6]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [7]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='3075fea6-82af-4b68-8d6d-583d418f061a'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Q0688Nzw1RiH63YOipy5r3ZA', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 150, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIBexlP7Zjpr0FwmIJfp80utYu9sq', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8c2bcf4a-be6c-416c-a166-b4a867261b97-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '

In [8]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='3075fea6-82af-4b68-8d6d-583d418f061a'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Q0688Nzw1RiH63YOipy5r3ZA', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 150, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIBexlP7Zjpr0FwmIJfp80utYu9sq', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8c2bcf4a-be6c-416c-a166-b4a867261b97-0', tool_calls=[{'name': 'tavily_search_results_json'

In [9]:
abot.graph.get_state(thread).next

('action',)

In [17]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='2eb20245-0bbe-4dd7-b545-dbeafeacb8f8'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_E7EEADJ5n9waxtbDIUmFrp7V', 'function': {'arguments': '{"query":"Los Angeles weather today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 150, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIBlearL0ksksLR3sWdOplQnh53w7', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7e804d71-40ea-45bb-bede-7671f0465e46-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Los Ang

In [18]:
while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread),"\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)


 StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='2eb20245-0bbe-4dd7-b545-dbeafeacb8f8'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_E7EEADJ5n9waxtbDIUmFrp7V', 'function': {'arguments': '{"query":"Los Angeles weather today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 150, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIBlearL0ksksLR3sWdOplQnh53w7', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7e804d71-40ea-45bb-bede-7671f0465e46-0', tool_calls=[{'name': 'tavily_search_results_json', 'ar

Modify state

In [19]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "4"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='b6143d9e-c683-44b6-af23-b49e34215de2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eaMk8qDevR4wWwlo4lXUl3MW', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 150, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIBq4ILRLV9f1wUckictE6ewtNFym', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-29ca967b-403e-4134-bddf-a4c7183aa803-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'cu

In [20]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='b6143d9e-c683-44b6-af23-b49e34215de2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eaMk8qDevR4wWwlo4lXUl3MW', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 150, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIBq4ILRLV9f1wUckictE6ewtNFym', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-29ca967b-403e-4134-bddf-a4c7183aa803-0', tool_calls=[{'name': 'tavily_search_results_json', 

In [22]:
current_values = abot.graph.get_state(thread)

In [23]:
current_values.values['messages'][-1]

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eaMk8qDevR4wWwlo4lXUl3MW', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 150, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIBq4ILRLV9f1wUckictE6ewtNFym', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-29ca967b-403e-4134-bddf-a4c7183aa803-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_eaMk8qDevR4wWwlo4lXUl3MW', 'type': 'tool_call'}], usage_metadata={'input_tokens': 150, 'output_tokens': 23,

In [24]:
current_values.values['messages'][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in Los Angeles'},
  'id': 'call_eaMk8qDevR4wWwlo4lXUl3MW',
  'type': 'tool_call'}]

In [26]:
_id = current_values.values['messages'][-1].tool_calls[0]['id']

current_values.values['messages'][-1].tool_calls = [
    {'name': 'tavily_search_results_json',
    'args': {'query': 'current weather in Louisiana'},
    'id': _id}
]

In [27]:
abot.graph.update_state(thread, current_values.values)

{'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='b6143d9e-c683-44b6-af23-b49e34215de2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eaMk8qDevR4wWwlo4lXUl3MW', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 150, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIBq4ILRLV9f1wUckictE6ewtNFym', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-29ca967b-403e-4134-bddf-a4c7183aa803-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'cu

{'configurable': {'thread_id': '4',
  'checkpoint_ns': '',
  'checkpoint_id': '1f01074f-16f7-694b-8002-c72c91da8558'}}

In [28]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='b6143d9e-c683-44b6-af23-b49e34215de2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eaMk8qDevR4wWwlo4lXUl3MW', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 150, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIBq4ILRLV9f1wUckictE6ewtNFym', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-29ca967b-403e-4134-bddf-a4c7183aa803-0', tool_calls=[{'name': 'tavily_search_results_json', 

In [29]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_eaMk8qDevR4wWwlo4lXUl3MW', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'title\': \'Weather in Louisiana\', \'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Louisiana\', \'region\': \'Missouri\', \'country\': \'USA United States of America\', \'lat\': 39.4411, \'lon\': -91.0551, \'tz_id\': \'America/Chicago\', \'localtime_epoch\': 1743672222, \'localtime\': \'2025-04-03 04:23\'}, \'current\': {\'last_updated_epoch\': 1743671700, \'last_updated\': \'2025-04-03 04:15\', \'temp_c\': 12.2, \'temp_f\': 54.0, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/113.png\', \'code\': 1000}, \'wind_mph\': 6.0, \'wind_kph\': 9.7, \'wind_degree\': 262, \'wind_dir\': \'W\', \'pressure_mb\': 1012.0, \'pressure_in\': 29.88, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidit

Time Travel

In [30]:
states = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print('--')
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='b6143d9e-c683-44b6-af23-b49e34215de2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eaMk8qDevR4wWwlo4lXUl3MW', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 150, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIBq4ILRLV9f1wUckictE6ewtNFym', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-29ca967b-403e-4134-bddf-a4c7183aa803-0', tool_calls=[{'name': 'tavily_search_results_json', 

In [37]:
to_replay = states[-3]

In [38]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='b6143d9e-c683-44b6-af23-b49e34215de2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eaMk8qDevR4wWwlo4lXUl3MW', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 150, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIBq4ILRLV9f1wUckictE6ewtNFym', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-29ca967b-403e-4134-bddf-a4c7183aa803-0', tool_calls=[{'name': 'tavily_search_results_json', 

In [39]:
for event in abot.graph.stream(None, to_replay.config):
    for k, v in event.items():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_eaMk8qDevR4wWwlo4lXUl3MW', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'title\': \'Thursday, April 3, 2025. Los Angeles, CA - Weather Forecast\', \'url\': \'https://weathershogun.com/weather/usa/ca/los-angeles/451/april/2025-04-03\', \'content\': \'Los Angeles, California Weather: Thursday, April 3, 2025. Cloudy weather, overcast skies with clouds. Day 66°. Night 50°. Precipitation 10 %.\', \'score\': 0.94314384}, {\'title\': \'Weather in Los Angeles\', \'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.0522, \'lon\': -118.2428, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1743676056, \'localtime\': \'2025-04-03 03:27\'}, \'current\': {\'last_updated_epoch\': 1743675300, \'last_updated\': \'2025-04-03

Add message to a state at a given time

In [40]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='b6143d9e-c683-44b6-af23-b49e34215de2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eaMk8qDevR4wWwlo4lXUl3MW', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 150, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIBq4ILRLV9f1wUckictE6ewtNFym', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-29ca967b-403e-4134-bddf-a4c7183aa803-0', tool_calls=[{'name': 'tavily_search_results_json', 

In [41]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']

In [ ]:
state_update = {"messages": [ToolMessage(
    tool_call_id=_id,
    name="tavily_search_results_json",
    content="54 degree celcius", # Changing message
)]}

In [43]:
branch_and_add = abot.graph.update_state(
    to_replay.config, 
    state_update, 
    as_node="action")

In [44]:
for event in abot.graph.stream(None, branch_and_add):
    for k, v in event.items():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='b6143d9e-c683-44b6-af23-b49e34215de2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eaMk8qDevR4wWwlo4lXUl3MW', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 150, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIBq4ILRLV9f1wUckictE6ewtNFym', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-29ca967b-403e-4134-bddf-a4c7183aa803-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'cu